In [12]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import numpy as np


### Plan

- write function to get soup from each page (i.e. 10 results) [x]
    - for each article, extract: title, link, year, site, first author [x]
    - NB if returns empty page, stop the for loop
- do this for 3 links: MRz, MRs, BC
    - exclude all articles that are not in BC (all papers citing THE paper but are not actually on BC)

In [17]:
def scrape_url(url):
    # get header: http://myhttpheader.com/
    headers = {'User-Agent':
           'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.115 Safari/537.36'}

    seconds =  round((30-5)*np.random.random()+5)
    print("Sleeping {} seconds to avoid bot detection".format(seconds))
    time.sleep(seconds) #from 5 to 30 seconds
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content,'lxml')
    
    if "Our systems have detected unusual traffic from your computer network" in str(soup):
        print ("Google has blocked this computer for a short time because it has detected this scraping script.")
        break 
        
    df_list = parse_soup(soup)
    return df_list
    
def parse_soup(soup):
    df_list = []
    
    num_articles = len(soup.select('[data-lid]'))
    
    for item in soup.select('[data-lid]'):
        print(item)
        try:
            # extarct from HTML
            title = [item.select('h3')[0].get_text()]
            title = [x.replace('[HTML][HTML] ', "") for x in title]
            link = [item.select('a')[0]['href']]
            abstract = [item.select('.gs_rs')[0].get_text()]
            info = item.select('.gs_a')[0].get_text()

            # parse info
            info_split = x.split(" - ")
            first_author = [info_split[0].split(", ")[0]]
            journal = info_split[1].split(", ")
            website = [info_split[2]]
            
            if len(journal) == 2:
                journal_name = [journal[0]]
                journal_year = [journal[1]]
            else:
                journal_year = [journal[0]]
                journal_name = [None]
            
            article = [*title, *journal_year , *first_author, *website *journal_name, *link]
            article = [x.strip(' ') for x in article]
            article = [x.replace('\xa0', "") for x in article]

        except Exception as e:
            #raise e 
            print('==cant parse this soup')
            # trying to catch where this is failing
            #print(item)
            #print(item.select('.gs_a')[0].get_text())
            #info_split = info.split("- ")
            #print(info_split)
            article=[]
            
        df_list.append(article)  
          
    return(df_list)
            
            

SyntaxError: 'break' outside loop (<ipython-input-17-be2018a3aed3>, line 18)

In [18]:
# test one
url='https://scholar.google.com/scholar?start=20&q=mendelian+randomization&hl=en&as_sdt=2005&sciodt=0,5&cites=6613969377995102680&scipsc=1'
df_list = scrape_url(url)
df_list
#df = pd.DataFrame(df_list, columns=['title', 'year', 'first_author', 'website', 'journal_name', 'link'])
#df

Sleeping 11 seconds to avoid bot detection


TypeError: exceptions must derive from BaseException

In [6]:
# scrape 10 pages by 10 results in each
total_df = pd.DataFrame()
for i in range(0, 100, 10):
    print("Results from {} to {}".format(i, i+9))
    url = 'https://scholar.google.com/scholar?start={}&q=mendelian+randomization&hl=en&as_sdt=2005&sciodt=0,5&cites=6613969377995102680&scipsc=1'.format(i)
    #print(url)
    df_list = scrape_url(url)
    df = pd.DataFrame(df_list, columns=['title', 'year', 'first_author', 'website', 'journal_name', 'link'])
    total_df = pd.concat([total_df,df])
total_df


Results from 0 to 9
Results from 10 to 19
Results from 20 to 29
Results from 30 to 39
Results from 40 to 49
Results from 50 to 59
Results from 60 to 69
Results from 70 to 79
Results from 80 to 89
Results from 90 to 99


,title,year,first_author,source_site,source_name,link


In [20]:

total_df.to_csv("parsing_MRz.csv")
